In [1]:
import pandas as pd
# pd.set_option('display.max_rows', 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import numpy as np
import os

In [2]:

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

labelDf = wr.s3.read_parquet(
    path=f"s3://partlist_label/",
    columns=["CaseID", "lvl_1_desc", "lvl_2_desc", "lvl_3_desc" ,"Description",	"fVAL",	"DamageCond"],
)
labelDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
0,12657878,bonnet,bonnet,bonnet,bonnet,550.00,bent/dented
1,12657888,bonnet,bonnet,bonnet,bonnet,510.44,bent
2,12657890,bonnet,bonnet,bonnet,bonnet,504.94,dented
3,12657900,bonnet,bonnet,bonnet,bonnet,450.00,crumpled
4,12657903,bonnet,bonnet,bonnet,bonnet,280.00,bent
...,...,...,...,...,...,...,...
6114698,12606104,rear panel inner trim,rear panel,rear panel,rear panel inner trim,235.00,broken
6114699,12606233,rear panel inner trim,rear panel,rear panel,rear panel inner trim,350.00,dented
6114700,12606480,rear panel inner trim,rear panel,rear panel,rear panel inner trim,250.00,broken
6114701,12606779,rear panel inner trim,rear panel,rear panel,firewall rear trim,225.00,


In [3]:
tempDf = labelDf[labelDf["CaseID"] == 10053736]
tempDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
130318,10053736,bonnet,bonnet,bonnet,bonnet,282.85,
422199,10053736,bumper front cover,bumper front,bumper front,bumper front cover,222.98,
1100382,10053736,headlamp assy rh,headlamp rh,headlamp rh,headlamp assy rh,373.74,
1677745,10053736,grille front,grille,grille,grille front,131.90,
3349762,10053736,fender front rh,fender front rh,fender front rh,fender front rh,100.80,


In [4]:
caseStudyDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_15/case_study_gt.csv")
caseStudyCaseId = caseStudyDf["CaseID"].tolist()


In [5]:
labelDf = labelDf[labelDf["CaseID"].isin(caseStudyCaseId)]
labelDf.sort_values(by=["CaseID", "lvl_3_desc"], inplace=True)
labelDf

/tmp/ipykernel_327757/4094135370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labelDf.sort_values(by=["CaseID", "lvl_3_desc"], inplace=True)


,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
60789,10021191,bonnet,bonnet,bonnet,bonnet,549.1,
317010,10021191,bumper front cover,bumper front,bumper front,bumper front cover,392.5,
1419393,10021191,air conditioning condenser,air conditioning,engine,air conditioning condenser,537.2,
853208,10021191,headlamp assy lh,headlamp lh,headlamp lh,headlamp assy lh,708.3,
2081154,10056498,bumper rear cover,bumper rear,bumper rear,bumper rear cover,308.1,cut/torn
...,...,...,...,...,...,...,...
3582286,13376628,airbag ecu,airbag,non external,airbag ecu,390.2,deployed
3636155,13376628,airbag passenger side,airbag,non external,airbag passenger side,634.9,deployed
3676535,13376628,seat belt reel front lh,interior,non external,seat belt reel front lh,369.0,jammed
3708425,13376628,seat belt reel front rh,interior,non external,seat belt reel front rh,369.0,jammed


In [6]:
outputDir = "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_15/partlist"
os.makedirs(outputDir, exist_ok=True)

In [7]:
import plotly.graph_objects as go
for caseId in labelDf["CaseID"].unique().tolist():
    df = labelDf[labelDf["CaseID"] == caseId]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),                line_color='darkslategray',
),
        cells=dict(values=[df[x] for x in df.columns],                line_color='darkslategray',
)
    )])

    fig.update_layout(width=1200, height=( np.clip(
        
            (len(df) // 4)* 300, 
            a_min=400, a_max=3000
            )
            )  
            )

# fig.show()
    fig.write_image(f"{outputDir}/{caseId}.png") 


In [8]:
multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df_lvl_3/",
        dataset=True,
    )


In [9]:
allTargetCol = [
 "headlamp_rh",'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
]

In [10]:
temp = multilabelDf[multilabelDf["CaseID"] == 12393479]
temp[allTargetCol]

KeyError: "['airbag', 'undercarriage', 'door_mirror_rh', 'interior', 'fog_lamp_lh', 'door_mirror_lh'] not in index"